# 线性回归的简洁实现

通过使用深度学习框架来简洁地实现
线性回归模型
生成数据集

In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import mindspore
import mindspore.dataset as ds
from d2l import mindspore as d2l

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

调用框架中现有的API来读取数据

In [3]:
def load_array(data_arrays, column_names, batch_size, is_train=True):  
    """构造一个MindSpore数据迭代器。"""
    dataset = ds.GeneratorDataset(data_arrays, column_names, shuffle=is_train)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 10
dataset = load_array(zip(features, labels), ['features', 'labels'], batch_size)

next(iter(dataset.create_tuple_iterator()))

[Tensor(shape=[10, 2], dtype=Float32, value=
 [[ 8.51511881e-02, -3.25021654e-01],
  [-1.33117318e+00, -1.25770819e+00],
  [-3.45325400e-03, -9.79733467e-01],
  ...
  [ 2.16835260e-01,  8.35646749e-01],
  [ 7.26829946e-01, -1.49664378e+00],
  [-2.02917647e+00, -8.47443104e-01]]),
 Tensor(shape=[10, 1], dtype=Float32, value=
 [[ 5.46032476e+00],
  [ 5.79959869e+00],
  [ 7.52748537e+00],
  ...
  [ 1.79961002e+00],
  [ 1.07523937e+01],
  [ 3.02442169e+00]])]

使用框架的预定义好的层, 这里初始化模型参数可以直接使用传参的方式

In [4]:
import mindspore.nn as nn
from mindspore.common.initializer import Normal

net = nn.SequentialCell([nn.Dense(2, 1, weight_init=Normal(0.01, 0), bias_init='zero')])

计算均方误差使用的是`MSELoss`类，也称为平方$L_2$范数

In [5]:
loss = nn.MSELoss()

将net与loss连接,这里使用MindSpore自带的Wrapper, `nn.WithLossCell`

In [6]:
net_with_loss = nn.WithLossCell(net, loss)

实例化一个`SGD`实例

In [7]:
optim = nn.SGD(net.trainable_params(), learning_rate=0.03)

将net,loss,optim连接，这里使用MindSpore自带的Wrapper, `nn.TrainOneStepCell`

In [8]:
trainer = nn.TrainOneStepCell(net_with_loss, optim)

In [9]:
dataset.children

训练过程代码与我们从零开始实现时所做的非常相似

In [10]:
num_epochs = 3
epoch = 0
for epoch in range(num_epochs):
    for X, y in dataset:
        l = trainer(X ,y)
    l = net_with_loss(mindspore.Tensor(features), mindspore.Tensor(labels))
    print(f'epoch {epoch + 1}, loss {l.asnumpy():f}')
# 这里报了Warning提示数据找不到，应该是bug，等待mindspore方面修复

[WARNING] ME(3050485:139974324291392,MainProcess):2021-10-30-02:46:04.340.190 [mindspore/dataset/engine/iterators.py:137] No records available.
[WARNING] ME(3050485:139974324291392,MainProcess):2021-10-30-02:46:04.347.666 [mindspore/dataset/engine/iterators.py:137] No records available.


epoch 1, loss 0.000310
epoch 2, loss 0.000310
epoch 3, loss 0.000310


比较生成数据集的真实参数和通过有限数据训练获得的模型参数

In [11]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

w的估计误差： [Tensor(shape=[], dtype=Float32, value= 0.00407457)
 Tensor(shape=[], dtype=Float32, value= -0.00747585)]
b的估计误差： [0.0112133]
